In [3]:
# Download and Extract Cifar-10 batch files
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xvzf cifar-10-python.tar.gz

--2025-07-20 06:57:21--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  75.4MB/s    in 2.2s    

2025-07-20 06:57:23 (75.4 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [5]:
# import packages
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [10]:
# Open and load batch-1 from cifar dataset
with open('cifar-10-batches-py/data_batch_1', 'rb') as fo:
    batch = pickle.load(fo, encoding='bytes')
# show batch keys
print(batch.keys())

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])


In [17]:
# Extract data and labels
data = batch[b'data']
labels = batch[b'labels']

In [ ]:
# In Cifar-10 dataset
# 0: "airplane", 1: "automobile", 2: "bird", 3: "cat", 4: "deer", 5: "dog", 6: "frog", 7: "horse", 8: "ship", 9: "truck"

In [26]:
# find 20 images for birds, cats, deers, dog and horse
bird_image = []
cat_image = []
deer_image = []
dog_image = []
horse_image = []

for img, lbl in zip(data, labels):
    if lbl == 2 and len(bird_image) != 20:
        bird_image.append(img)
    if lbl == 3 and len(cat_image) != 20:
        cat_image.append(img)
    if lbl == 4 and len(deer_image) != 20:
        deer_image.append(img)
    if lbl == 5 and len(dog_image) != 20:
        dog_image.append(img)
    if lbl == 7 and len(horse_image) != 20:
        horse_image.append(img)

    if len(bird_image) == 20 and len(cat_image) == 20 and len(deer_image) == 20 and len(dog_image) == 20 and len(horse_image) == 20:
        break

print("Number of bird images", len(bird_image))
print("Number of cat images", len(cat_image))
print("Number of deer images", len(deer_image))
print("Number of dog images", len(dog_image))
print("Number of horse images", len(horse_image))

Number of bird images 20
Number of cat images 20
Number of deer images 20
Number of dog images 20
Number of horse images 20


In [38]:
# Equally distribute the classes inside the new 100 image dataset
# New labels
# 0: "bird", 1: "cat", 2: "deer", 3: "dog", 4: "horse"

img_dataset = []

for bird, cat, deer, dog, horse in zip(bird_image, cat_image, deer_image, dog_image, horse_image):
    img_dataset.append({"image": bird, "label": 0})
    img_dataset.append({"image": cat, "label": 1})
    img_dataset.append({"image": deer, "label": 2})
    img_dataset.append({"image": dog, "label": 3})
    img_dataset.append({"image": horse, "label": 4})

print(len(img_dataset))

100
